# 5-Body Simulation of the Inner Solar System

## Objective
This project creates a physically accurate mathematical model and subsequent interactive animations of the orbits of the inner solar system. I started with position and velocity data from the NASA JPL horizons system and applied Newton's Law of Universal Gravitation in a way that represents the orbital mechanics. This resulted in both a 2D animated video and a fully interactive 3D model.

### Skills Demonstrated
**Programming:** Python
**Libraries:** NumPy, Matplotlib, Plotly
**Techniques:** N-Body Simulation, Numerical Integration (Leapfrog), 2D/3D Animation, Data Sourcing (NASA JPL), Scientific Visualization

## Data Sourcing
Initial position and velocity vectors of the inner solar system planets and the sun were obtained from [NASA's JPL website](https://ssd.jpl.nasa.gov/horizons/app.html#/) using the starting time of January 1, 2024 at 00:00.

In [ ]:
import numpy as np

#gravitation constant in km^3 kg^-1 day^-2
G = 4.982057e-10

# creating parameters in days
time_step = 0.01
total_time = 700

# Celestial bodies data structure
bodies = [
    {
        "name": "Sun",
        "mass": 1.98847e30, # kg
        "pos": np.array([-1.191989205285392E+06, -4.347654370292560E+05, 3.146124454779254E+04]), 
        "vel": np.array([8.441016159655652E-03, -1.221912155158748E-02, -7.918737699351246E-05]) * 86400 
    },
    {
        "name": "Mercury",
        "mass": 0.33011e24,
        "pos": np.array([-4.227610797568034E+07, 2.953899410451555E+07, 6.249351652770508E+06]), 
        "vel": np.array([-3.864898908767687E+01, -3.735110987200028E+01, 4.943644151005024E-01]) * 86400 
    },
    {
       "name": "Venus",
        "mass": 4.8675e24,
        "pos": np.array([-1.081907314450878E+08, -1.189049058816830E+07, 6.048049571687455E+06]), 
        "vel": np.array([3.521901293154280E+00, -3.498977541526820E+01, -6.831705083215418E-01]) * 86400
    },
    {
        "name": "Earth",
        "mass": 5.9722e24,
        "pos": np.array([-2.600745330322259E+07, 1.445621192952758E+08, 2.355195999945700E+04]), 
        "vel": np.array([-2.983800155786667E+01, -5.149120788710330E+00, 3.849310062455924E-04]) * 86400 
    },
    {
        "name": "Mars",
        "mass": 0.64171e24,
        "pos": np.array([-4.507776377918092E+07, -2.175196919117580E+08, -3.441546039979607E+06]), 
        "vel": np.array([2.467035556790299E+01, -2.734379316488365E+00, -6.620611008282651E-01]) * 86400 
    }
]

def calculate_acceleration(bodies, G):
    #create an array of zeroes to store accelerations
    accelerations = np.zeros((len(bodies), 3))

    # create a loop to go through each body and calculate the total force on it
    for i, body_i in enumerate(bodies):
        # second loop through bodies to calculate individual gravities effects on acceleration
        for j, body_j in enumerate(bodies):
           if i == j:
               continue #this is because when i=j gravitational force is undefined since r=0
           # calculate the r vector
           r_ij = body_j["pos"] - body_i["pos"]
           # use the Pythagorean theorem in 3d to calculate the distance between the bodies
           dist = np.linalg.norm(r_ij)
           # calculate acceleration using newton's law of universal gravitation m1a=Gm1m2/r^2 or a=Gm2/r^2
           accel_magnitude = G * body_j["mass"]/dist**3
           accel_vector = accel_magnitude * r_ij

           # add acceleration to the acceleration array for this body
           accelerations[i] += accel_vector
    return accelerations

# prepare a list to store the history of positions for plotting later
# create 3D array: (num_bodies, num_timesteps, 3 for x,y,z
num_steps = int(total_time/time_step)
positions_history = np.zeros((len(bodies), num_steps + 1, 3))

#store the initial positions in the history array
for i, body in enumerate(bodies):
    positions_history[i, 0] = body["pos"]

# Calculate the initial acceleration
accelerations = calculate_acceleration(bodies, G)

## Simulation Method
After calculating the acceleration using Newton's Law of Universal Gravitation between each body and summing them up into the accelerations vector, I used Leapfrog integration to calculate the positions at each time step and stored them in a positions_history array. I chose this method due to its excellent long-term stability in orbital mechanics problems. As a symplectic integrator, it excels at conserving the total energy of the system which prevents the orbits from spiraling out of control in the short time of this simulation. The positions_history array provided me with the data points required to run the 2D and 3D representations.

In [ ]:
# Run the simulation using leapfrog integration
for step in range(num_steps):
    for i, body in enumerate(bodies):
        body["vel"] += accelerations[i] * (time_step/2)

    for i, body in enumerate(bodies):
        body["pos"] += body["vel"] * time_step
        #store the new position in the history log
        positions_history[i, step + 1] = body["pos"]

    #calculate new accelerations
    accelerations = calculate_acceleration(bodies, G)

    #updated velocities by final half step
    for i, body in enumerate(bodies):
        body["vel"] += accelerations[i] * (time_step / 2)

print("Simulation finished.")

## 2D Interactive Animation (Matplotlib)
This first plot is a 2D top down view of the solar system. It incorporates the x and y data points to illustrate the shapes of the orbits. The markers moving around respresent each planet as the data is processed over the 700 day period the simulation covers.

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
# creating a 2D top down simulation of the inner solar system.
# Set up the plot
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(10, 10))

#setting the limits to just outside the orbit of mars
ax.set_xlim(-2.5e8, 2.5e8)
ax.set_ylim(-2.5e8, 2.5e8)

#update labels
ax.set_title("Animated Inner Solar System Orbits")
ax.set_xlabel("X Position (km)")
ax.set_ylabel("Y Position (km)")
ax.set_aspect('equal', adjustable='box')
ax.grid(True, linestyle='--', alpha=0.5)

colors = ['yellow', 'orange', 'ivory', 'blue', 'red']

#creates a starting point for the lines of the display
lines = [ax.plot([], [], label=body["name"], color=color)[0] for body, color in zip(bodies, colors)]
points = [ax.plot([], [], marker='o', color=lines[i].get_color())[0] for i, body in enumerate(bodies)]

def update(frame):
    # Loop through planets
    for i, (line, point) in enumerate(zip(lines, points)):
        line.set_data(positions_history[i, :frame, 0], positions_history[i, :frame, 1])
        point.set_data([positions_history[i, frame, 0]], [positions_history[i, frame, 1]])
    return lines + points

#calculate the interval needed for a 30s animation in ms
interval_ms = 5000/num_steps
# create the animation object
ani = FuncAnimation(
    fig=fig,
    func=update,
    frames=num_steps,
    interval=interval_ms,
    blit=False
)

ax.legend()


## 3D Interactive Animation (Plotly)
For a more comprehensive view and to illustrate the full movement of the planets I created a 3D interactive animation using Plotly to use WebGL for smooth, GPU-accelerated rendering which allows for real-time data exploration. 

### Key Features & Findings:

- **Exploration Mode:** This animation allows for interactive exploration. You can click and drag the model and scroll in and out to view the solar system from any imaginable angle
- **Animated Mode:** the Play/Pause buttons and the time slide allow for control of the simulation allowing the simulation to be played and paused anywhere in the 700 day period. 
- **Coplanar Orbits:** By rotating the system, the nearly coplanar nature of the solar system is easily seen.
- **Orbital Inclination:** You can also view the way the orbits of the planets are not in perfect planar alignment which is especially visible with Mercury and Venus.

In [ ]:
import plotly.graph_objects as go
#import plotly.io as pio

#pio.renderers.default = "notebook"

fig = go.Figure()
colors = ['yellow', 'orange', 'ivory', 'blue', 'red']

# creates the orbital lines
for i, (body, color) in enumerate(zip(bodies, colors)):
    fig.add_trace(
        go.Scatter3d(
            x=positions_history[i, :, 0],
            y=positions_history[i, :, 1],
            z=positions_history[i, :, 2],
            mode='lines',
            line=dict(color=color, width=4),
            showlegend=False
    ))
        #creates the planetary markers that will be animated
for i, (body, color) in enumerate(zip(bodies, colors)):
    fig.add_trace(
        go.Scatter3d(
            x=[positions_history[i, 0, 0]],
            y=[positions_history[i, 0, 1]],
            z=[positions_history[i, 0, 2]],
            mode='markers',
            name=body["name"],
            marker=dict(color=color, size=5)
    ))

# evenly spaces 200 frames for the animation
num_animation_frames = 200
steps_to_render = np.linspace(0, num_steps, num_animation_frames, dtype=int)

frames = []

#creates each frame
for step in steps_to_render:
    frame=go.Frame(
        data=[
            go.Scatter3d(
                x=[positions_history[i, step, 0]],
                y=[positions_history[i, step, 1]],
                z=[positions_history[i, step, 2]],
            ) for i in range(len(bodies))
        ],
        traces=[5,6,7,8,9],
        name=f"Day {int(step * time_step)}"
    )
    frames.append(frame)

#Add the frames to the figure
fig.frames = frames

# update axis titles and simulation title
fig.update_layout(
    title_text="Interactive 3D Simulation of Inner Solar System Orbits",
    template="plotly_dark",
    width=800,
    height=800,
    scene=dict(
        xaxis_title="X (km)", yaxis_title="Y (km)", zaxis_title="Z (km)",
        aspectmode='data'
    ),
    # Add play button
     updatemenus=[dict(
        type="buttons",
        showactive=False,
        y=0.95, 
        x=0.1,
        xanchor="left",
        yanchor="top",
        pad=dict(t=0, r=10),
        buttons=[
            # Play Button
            dict(
                label="▶ Play",
                method="animate",
                args=[None, {"frame": {"duration": 50}, "fromcurrent": True, "transition": {"duration": 0}, "mode": "immediate"}]
            ),
            # Pause Button
            dict(
                label="❚❚ Pause",
                method="animate",
                args=[[None], {"frame": {"duration": 0}, "mode": "immediate", "transition": {"duration": 0}}]
            )
        ]
                      
    )],
    # Add a time slider
    sliders=[dict(
        steps=[dict(method='animate',
                    args=[[f"Day {int(s*time_step)}"], {"frame": {"duration": 50, "redraw": False}, "mode": "immediate"}],
                    label=str(int(s*time_step))) for s in steps_to_render]
    )]
)

fig.show()

## Conclusion
In summary, this project successfully demonstrated the creation of a physically accurate model of the inner solar system using initial data sourced from the NASA JPL horizons system. The model produced stable and realistic orbits that accurately display the paths of all four planets and revealed the nearly coplanar aspect of the solar system.

Potential future work could include expanding the simulation to include the rest of the solar system and incorporate the gravitational effects of major moons and minor planets for higher precision or expanding the model to include relativistic effects.